In [1]:
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
from textblob import TextBlob 
from datetime import datetime
import requests

%run ./fair_keys.ipynb

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


url = "https://api.aiforthai.in.th/ssense"
headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

In [44]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022_1.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#df['Time'] = pd.to_datetime(df['Time'])
time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
df.loc[mask]

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
55,animation,SagarGa39569425,0.0,0.0,"The plot of ""Don't Mess With The Cultivation B...",2022-03-19,NaN,NaN,NaN,NaN,NaN
68,animation,Aarya_pimparkar,0.0,0.0,"I love the art style in ""Love After Marriage"" ...",2022-03-19,NaN,NaN,NaN,NaN,NaN
77,animation,jkhomezi,0.0,0.0,"You'll find companies looking for a ""Graphic D...",2022-03-19,NaN,NaN,NaN,NaN,NaN
165,animation,blazingwish66,0.0,0.0,"The plot of ""Martial Rebel"" is like a roller c...",2022-03-19,NaN,NaN,NaN,NaN,NaN
167,animation,DI0RDENKI,0.0,0.0,I think diluc should get a new idle animation ...,2022-03-19,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1209,อนิเมะแนะนำ,tinyreview2,37.0,0.0,RT bbrinkk bilibili,2022-03-21,th,ประเทศไทย,['สาววายถ'],positive,97.30
1220,อนิเมะแนะนำ,cara_mellox,2.0,0.0,RT btfeffectx 86 EIGHTYSIX,2022-03-21,th,รักพี่บาจิเหมือนหมา 4ever,[],negative,66.67
1221,อนิเมะแนะนำ,btf_effectx,2.0,1.0,86 EIGHTYSIX Attack on Titan,2022-03-21,th,"she/her, th-en ♡",[],positive,83.33
1222,อนิเมะแนะนำ,mydissocute,2.0,0.0,RT btfeffectx 86 EIGHTYSIX,2022-03-21,th,NaN,[],negative,66.67


In [17]:
# cursor = tw.Cursor(api.user_timeline,
#                             id='Fa6i7r2',
#                             tweet_mode="extended",
#                             since = '2022-03-17',
#                             include_entities=True).items(10)

cursor = tw.Cursor(api.search_tweets,
                            id='Fa6i7r2',
                            tweet_mode="extended",
                            since = '2022-03-17',
                            include_entities=True).items(10)
for tweet in cursor:
    #if "RT @" not in tweet.full_text:
    print([tweet.full_text,tweet.retweet_count,tweet.favorite_count])
    print(str(tweet.created_at)) 


Unexpected parameter: id
Unexpected parameter: since
Unexpected parameter: include_entities


['Check twitter api status', 1, 1]
2022-03-24 07:47:22+00:00


In [2]:
today = datetime.today()
#filename = str("tweet_data_"+str(today.day)+str(today.month)+str(today.year)+".csv")
filename = 'tweet_data_2032022.csv'
df = pd.read_csv(filename)

,Keyword,User,Retweet,Likes,Tweet,Time
0,3d,Dominom_,0.0,0.0,also next week is my last week of this class :...,2022-03-21 06:27:16+00:00
1,3d,jannis_3d,0.0,0.0,@GriffinWB Butterflies you mean MOTH https://t...,2022-03-21 06:27:21+00:00
2,animation,boringbambi,0.0,0.0,Netflix needs to make more animation and kdrama,2022-03-20 14:13:00+00:00
3,animation,depressedcule99,0.0,0.0,ShinobusMyGoat Ratio doenst work because im Ri...,2022-03-20 15:14:30+00:00
4,animation,NiiTawiah2,0.0,0.0,The plot of The Supreme Expert is like a rolle...,2022-03-20 14:43:23+00:00
...,...,...,...,...,...,...
3476,sport anime,AliffMustafa180,1.0,1.0,aimannadzmi1 LZJ is an MC from a sport anime,2022-03-19 22:11:35+00:00
3477,sport anime,yueme0,0.0,0.0,Im literally hooked on this sport anime Its ab...,2022-03-19 20:59:34+00:00
3478,sport anime,tsukinagass,0.0,0.0,THE OTHER ADULT SPORT ANIME IVE WATCHED IS ONE...,2022-03-20 05:22:46+00:00
3479,sport anime,kaitou_m,0.0,0.0,"On the sports anime side of things, this is wh...",2022-03-21 04:38:17+00:00


In [6]:
def getSentiment(text):
    # set sentiment
    if TextBlob(text).sentiment.polarity > 0:
        return 'positive'
    elif TextBlob(text).sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

def extract_hashtags(text):
    regex = "#(\w+)" 
    hashtag_list = re.findall(regex, text)
    return hashtag_list

def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [7]:
def get_related_tweets(key_word,date):
    tweet_keyword = []
    twitter_users = []
    twitter_users_location = []
    tweet_hashtag = []
    tweet_time = []
    tweet_string = [] 
    tweet_countRT = []
    tweet_fav = []
    tweet_sentiment = []
    tweet_polarity = []
    tweet_language = []
    if date != 'now':
        cursor = tw.Cursor(api.search_tweets,
                            q=key_word,
                            tweet_mode="extended",
                            since = date,
                            include_entities=True).items(30)
    else:
        cursor = tw.Cursor(api.search_tweets,
                            q=key_word,
                            tweet_mode="extended",
                            include_entities=True).items(30)
    for tweet in cursor:
                            
        if(tweet.lang == 'en' or tweet.lang == 'th'):
            twitter_users.append(tweet.user.screen_name)
            twitter_users_location.append(tweet.user.location)
            tweet_time.append(tweet.created_at)
            tweet_string.append(remove_url(tweet.full_text))
            tweet_countRT.append(tweet.retweet_count)
            tweet_fav.append(tweet.favorite_count)
            tweet_keyword.append(key_word)
            tweet_hashtag.append(str(extract_hashtags(tweet.full_text)))
            tweet_language.append(tweet.lang)
            if tweet.lang == 'en':
                tweet_polarity.append(getSentiment(tweet.full_text))
                tweet_sentiment.append(TextBlob(tweet.full_text).sentiment.polarity)
            elif tweet.lang == 'th':
                text = re.sub(r'[%]',' ',tweet.full_text)
                params = {'text':text}
                response = requests.get(url, headers=headers, params=params)
                try:
                    polarity = str(response.json()['sentiment']['polarity'])
                    sentiment = str(response.json()['sentiment']['score'])
                except (KeyError):
                    polarity = 'neutral'
                    sentiment = 0
                tweet_polarity.append(polarity)
                tweet_sentiment.append(sentiment)

    df = pd.DataFrame({'Keyword':tweet_keyword,'User':twitter_users,'Tweet': tweet_string,'Language':tweet_language, 'Time': tweet_time,'User Location':twitter_users_location,
                        'Hashtag':tweet_hashtag,'Polarity':tweet_polarity,'Likes':tweet_fav,'Retweet':tweet_countRT,'Sentiment':tweet_sentiment})

    return df

In [ ]:
get_related_tweets(key_word,date)

In [5]:
for tweet in tw.Cursor(api.search_tweets,q='Genshin Impact',tweet_mode="extended",
                        include_entities=True,lang="en",since='2022-01-17').items(10):
    print('\n\nuser',tweet.user.screen_name)
    print('create',tweet.created_at)
    print('tweet',tweet.full_text)
    print('RT count',tweet.retweet_count)
    print('tweet fav',tweet.favorite_count)
    print('hashtag',extract_hashtags(tweet.full_text))
    print('language',tweet.lang)
    if tweet.lang == 'en':
        print('polarity',getSentiment(tweet.full_text))
        print('sentiment',TextBlob(tweet.full_text).sentiment.polarity)
    elif tweet.lang == 'th':
        text = tweet.full_text
        params = {'text':text}
        response = requests.get(url, headers=headers, params=params)
        print('polarity',response.json()['sentiment']['polarity'])
        print('sentiment',response.json()['sentiment']['score'])

Unexpected parameter: since




user hibouxz
create 2022-03-24 07:21:26+00:00
tweet RT @litarnes: Reminders of home

Childe ( Genshin Impact ) 原神 https://t.co/hOy0Q5BY6A
RT count 7651
tweet fav 0
hashtag []
language en
polarity neutral
sentiment 0.0


user _berryrandom
create 2022-03-24 07:21:21+00:00
tweet RT @HoYoLAB_Mimo: 📘HoYoLAB's new content, "HoYoLAB Magazine," is now online!

✨This issue contains fantastic fan works of Genshin Impact, H…
RT count 490
tweet fav 0
hashtag []
language en
polarity positive
sentiment 0.2852272727272727


user JosRd19
create 2022-03-24 07:21:19+00:00
tweet RT @racheltjhia: does anyone still play Genshin Impact? https://t.co/kjY6gTDoq9
RT count 686
tweet fav 0
hashtag []
language en
polarity neutral
sentiment 0.0


user modrasolata
create 2022-03-24 07:21:14+00:00
tweet RT @litarnes: Reminders of home

Childe ( Genshin Impact ) 原神 https://t.co/hOy0Q5BY6A
RT count 7650
tweet fav 0
hashtag []
language en
polarity neutral
sentiment 0.0


user I_Leak_VN
create 2022-03-24 07:21:11+00:0